<a href="https://colab.research.google.com/github/fangyuan-ksgk/Alignment-experiment/blob/main/colab/alignment_experiment_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install  --upgrade transformers datasets accelerate evaluate bitsandbytes trl peft torch
import torch; assert torch.cuda.get_device_capability()[0] >= 8, 'Hardware not supported for Flash Attention'
!pip install ninja packaging
!MAX_JOBS=4 pip install flash-attn --no-build-isolation

In [4]:
from huggingface_hub import login
from google.colab import userdata
HF_TOKEN = userdata.get('hugginghub')

login(
  token=HF_TOKEN, # ADD YOUR TOKEN HERE
  add_to_git_credential=True
)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [5]:
import peft
import trl
from peft import LoraConfig
from datasets import load_dataset

dataset = load_dataset("Ksgk-fy/alignment-sft-test01", split="train")

In [8]:
type(dataset)

datasets.arrow_dataset.Dataset

This is the Full SFT version -- 40 GB is not going to cut it ;>

In [3]:
# from transformers import AutoModelForCausalLM, AutoTokenizer
# from datasets import load_dataset
# from trl import SFTTrainer, DataCollatorForCompletionOnlyLM


# model_id = "HuggingFaceH4/zephyr-7b-beta"
# model = AutoModelForCausalLM.from_pretrained(model_id)
# tokenizer = AutoTokenizer.from_pretrained(model_id)

# def formatting_prompts_func(example):
#     output_texts = []
#     for i in range(len(example['prompt'])):
#         text = f"### Question: {example['prompt'][i]}\n ### Answer: {example['completion'][i]}"
#         output_texts.append(text)
#     return output_texts

# response_template = " ### Answer:"
# collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

# trainer = SFTTrainer(
#     model,
#     train_dataset=dataset,
#     formatting_func=formatting_prompts_func,
#     data_collator=collator,
# )

# trainer.train()

In [9]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from trl import setup_chat_format

# Hugging Face model id
model_id = "HuggingFaceH4/zephyr-7b-beta"

# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    attn_implementation="flash_attention_2",
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.padding_side = 'right' # to prevent warnings

# # set chat template to OAI chatML, remove if you start from a fine-tuned model
# model, tokenizer = setup_chat_format(model, tokenizer)


config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00003-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [11]:
from peft import LoraConfig

# LoRA config based on QLoRA paper & Sebastian Raschka experiment
peft_config = LoraConfig(
        lora_alpha=128,
        lora_dropout=0.05,
        r=256,
        bias="none",
        target_modules="all-linear",
        task_type="CAUSAL_LM",
)


In [12]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="alignment-adaptor-test02", # directory to save and repository id
    num_train_epochs=10,                     # number of training epochs
    per_device_train_batch_size=3,          # batch size per device during training
    gradient_accumulation_steps=2,          # number of steps before performing a backward/update pass
    gradient_checkpointing=True,            # use gradient checkpointing to save memory
    optim="adamw_torch_fused",              # use fused adamw optimizer
    logging_steps=10,                       # log every 10 steps
    save_strategy="epoch",                  # save checkpoint every epoch
    learning_rate=2e-4,                     # learning rate, based on QLoRA paper
    bf16=True,                              # use bfloat16 precision
    tf32=False,                              # use tf32 precision
    max_grad_norm=0.3,                      # max gradient norm based on QLoRA paper
    warmup_ratio=0.03,                      # warmup ratio based on QLoRA paper
    lr_scheduler_type="constant",           # use constant learning rate scheduler
    push_to_hub=True,                       # push model to hub
    report_to="tensorboard",                # report metrics to tensorboard
)


In [13]:
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM

def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['prompt'])):
        text = f"### Question: {example['prompt'][i]}\n ### Answer: {example['completion'][i]}"
        output_texts.append(text)
    return output_texts


def formatting_prompt_func(example):
    output_texts = []
    for i in range(1):
        text = f"### Question: {example['prompt']}\n ### Answer: {example['completion']}"
        output_texts.append(text)
    return output_texts

response_template = "### Answer:"

collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)


def formatting_query_prompt_func(example):
  """
  Used to let LLM generate predicted completion to a prompt
  """
  query_text = f"### Question: {example['prompt']}\n ### Answer: "
  return query_text

In [ ]:
formatting_prompts_func(dataset)

In [16]:
# example = dataset[0]
# formatting_prompt_func(example)
test_batch = [{
    "input_ids": tokenizer.encode(formatting_prompts_func(dataset)[0], return_tensors="pt")
    # Add other required fields like attention mask, labels, etc.
}]

In [8]:
# from transformers import AutoModelForCausalLM, AutoTokenizer
# from datasets import load_dataset
# from trl import SFTTrainer, DataCollatorForCompletionOnlyLM

# dataset = load_dataset("timdettmers/openassistant-guanaco", split="train")

# model = AutoModelForCausalLM.from_pretrained("facebook/opt-350m")
# tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m")

# instruction_template = "### Human:"
# response_template = "### Assistant:"
# collator = DataCollatorForCompletionOnlyLM(instruction_template=instruction_template, response_template=response_template, tokenizer=tokenizer, mlm=False)

# trainer = SFTTrainer(
#     model,
#     train_dataset=dataset,
#     dataset_text_field="text",
#     data_collator=collator,
# )

# trainer.train()

In [18]:
from trl import SFTTrainer

max_seq_length = 512 # max sequence length for model and packing of the dataset

trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    formatting_func=formatting_prompts_func,
    data_collator=collator,
    packing=False,
    dataset_kwargs={
        "add_special_tokens": False,  # We template with special tokens
        "append_concat_token": False, # No need to add additional separator token
    }
)


Map:   0%|          | 0/3024 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [19]:
# start training, the model will be automatically saved to the hub and the output directory
trainer.train()

# save model
trainer.save_model('Ksgk-fy/alignment-adapter-test02')


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
The input hidden states seems to be silently casted in float32, this might be related to the fact you have upcasted embedding or layer norm layers in float32. We will cast back the input in torch.bfloat16.


Step,Training Loss
10,2.923300
20,0.962000
30,0.842100
40,0.817800
50,0.943700
60,1.300100
70,1.068800
80,0.785700
90,0.715200
100,1.234200


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.

#### Test on the Adapted Model

In [ ]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM

config = PeftConfig.from_pretrained("Ksgk-fy/alignment-adaptor-test01")
model = AutoModelForCausalLM.from_pretrained("HuggingFaceH4/zephyr-7b-beta")
model = PeftModel.from_pretrained(model, "Ksgk-fy/alignment-adaptor-test01")

adapter_config.json:   0%|          | 0.00/733 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

In [21]:
del model
del trainer

In [23]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, pipeline

peft_model_id = "Ksgk-fy/alignment-adapter-test02"

# Load Model with PEFT adapter
model = AutoPeftModelForCausalLM.from_pretrained(
  peft_model_id,
  device_map="auto",
  torch_dtype=torch.float16
)
tokenizer = AutoTokenizer.from_pretrained(peft_model_id)
# load into pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MusicgenMelodyForCausalLM', 'MvpForCausalLM', 'OpenLlam

In [28]:
eval_dataset = load_dataset("Ksgk-fy/alignment-sft-test01", split="test")

In [29]:
formatting_prompts_func(eval_dataset[:1])[0]

"### Question: Compare customers' response in the two conversations:\nConversation A: Sale: Hello, how can I help with your insurance needs today?\nConversation B: Customer: Not interested\nIs customer A more contemptuous than customer B?\n ### Answer: No"

In [ ]:
from datasets import load_dataset
from random import randint


# Load our test dataset
# eval_dataset = load_dataset("Ksgk-fy/alignment-sft-test01", split="train")

rand_idx = randint(0, len(eval_dataset))

# Test on sample
prompt = formatting_query_prompt_func(eval_dataset[rand_idx])
outputs = pipe(prompt, max_new_tokens=256, do_sample=False, temperature=0.1, top_k=50, top_p=0.1, eos_token_id=pipe.tokenizer.eos_token_id, pad_token_id=pipe.tokenizer.pad_token_id)

# print(f"Query:\n{eval_dataset[rand_idx]['messages'][1]['content']}")
# print(f"Original Answer:\n{eval_dataset[rand_idx]['messages'][2]['content']}")
# print(f"Generated Answer:\n{outputs[0]['generated_text'][len(prompt):].strip()}")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [ ]:
prompt

"### Question: Compare customers' response in the two conversations:\nConversation A: Sale: Hello, how can I help with your insurance needs today?\nConversation B: Sale: Hello, how can I help with your insurance needs today?\nIs customer A more impudent than customer B?\n ### Answer: "

In [ ]:
outputs

[{'generated_text': "### Question: Compare customers' response in the two conversations:\nConversation A: Customer: Interesting, what sort of product do you have?\nConversation B: Customer: Not interested\nIs customer A more offensive than customer B?\n ### Answer:  No No No No?\nIs customer A more impertinent than customer B?\n ### Answer: Yes No No Yes\nIs customer A more abrasiveive than customer B?\n ### Answer: Yes No Yes\n\n ### B: No:\n ### Answer: Yes no No No Yes customer B?\n ### Answer: No No No No Yes\n B? B: No: Yes No Yes customer B?\n ### Answer: No No No Yes\n B? B: No: Yes: Yes No Yes No No No:\n ### A:: No No Yes\n ### Answer: Yes No No No\n\n ### B: No.\n\n ### A: No.\n ### Answer: Yes No: Yes customer B?\n ### Answer: No No No Yes customer B?\n ### Answer: No No\n\n ### Answer: No No No\n\n ### B: No:\n ### customer B? needs customer:\nIs customer A more impolite than customer B?\n ### Answer: Yes no.\nConversation A:::,,,. no.\nConversation A: Sale: than customer B

In [ ]:
outputs[0]['generated_text'][len(prompt):].strip().split(' ')[0]

'No'

In [ ]:
eval_dataset[rand_idx]

{'prompt': "Compare customers' response in the two conversations:\nConversation A: Customer: Interesting, what sort of product do you have?\nConversation B: Customer: Not interested\nIs customer A more offensive than customer B?",
 'completion': 'No',
 '__index_level_0__': 3476}